### **1) REQUIREMENTS SETUP**### **1) REQUIREMENTS SETUP**# **Regression data preparation and modeling**

### **1) REQUIREMENTS SETUP**

In [ ]:
# !pip install -r requirements.txt

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from pathlib import Path
from linearmodels.panel import PanelOLS
from scipy.optimize import minimize

### **2) MODULES IMPORT**

In [ ]:
# None

### **3) DATA Prep**

In [2]:
# Normalization of all the data used in the regression over comparable timeframe / format
data_fetcher_path = Path.cwd().parent / "data_fetcher"
dep_IPI = pd.read_csv(data_fetcher_path/"aggregate_df/EURO_indprod_dependent_df.csv")
dep_Stocks = pd.read_csv(data_fetcher_path/"aggregate_df/EURO_stock_dependent_df.csv")

#Other (!!! I change the path to trade openness 1 from EURO_trade... to trade_openness_ ...)
exposure_df = pd.read_csv(data_fetcher_path / "country_tariff_exposure.csv")
openness_df = pd.read_csv(data_fetcher_path/"aggregate_df/trade_openness_annual_regime_df.csv")
controls_df = pd.read_csv(data_fetcher_path/"aggregate_df/country_specific_test_df.csv")
wb_openness_df = pd.read_csv(data_fetcher_path/"aggregate_df/wb_trade_openness_annual_regime_df.csv")
transition_df = pd.read_csv(data_fetcher_path/"aggregate_df/Export_Intra_EU.csv")


# === OFFICIAL EU COUNTRIES ONLY ===
eu_country_map = {
    "Austria": "AT", "Belgium": "BE", "Bulgaria": "BG", "Croatia": "HR",
    "Cyprus": "CY", "Czechia (Czech Republic)": "CZ", "Czechia": "CZ",
    "Denmark": "DK", "Estonia": "EE", "Finland": "FI", "France": "FR",
    "Germany": "DE", "Greece": "GR", "Hungary": "HU", "Ireland": "IE",
    "Italy": "IT", "Latvia": "LV", "Lithuania": "LT", "Luxembourg": "LU",
    "Malta": "MT", "Netherlands": "NL", "Poland": "PL", "Portugal": "PT",
    "Romania": "RO", "Slovakia": "SK", "Slovenia": "SI", "Spain": "ES",
    "Sweden": "SE"
}
EU_ISO_CODES = set(eu_country_map.values())

def keep_only_eu(df, country_col='Country'):
    if country_col not in df.columns:
        return df
    before = len(df)
    df = df[df[country_col].isin(EU_ISO_CODES)].copy()
    after = len(df)
    dropped = before - after
    if dropped:
        print(f"Dropped {dropped:,} non-EU rows → {after:,} EU rows kept")
    return df

# APPLY: Keep only EU countries
dep_IPI = keep_only_eu(dep_IPI)
dep_Stocks = keep_only_eu(dep_Stocks)
exposure_df = keep_only_eu(exposure_df)
openness_df = keep_only_eu(openness_df)
controls_df = keep_only_eu(controls_df)
transition_df = keep_only_eu(transition_df)
wb_openness_df = keep_only_eu(wb_openness_df)


print(f"\nKept only 27 EU countries: {sorted(EU_ISO_CODES)}\n")


start_date = '2024-11'
end_date = '2025-08'

# ================================
# IPI: Full Lagged First Difference (MAXIMUM COVERAGE)
# ================================

# 1. Start with FULL data (no date filter yet)
cols = ['Country', 'Level 1 Index', 'Time', 'Indprod Index Value (I21)']
dep_IPI_full = dep_IPI[cols].copy()

# 2. Ensure numeric
dep_IPI_full['Indprod Index Value (I21)'] = pd.to_numeric(
    dep_IPI_full['Indprod Index Value (I21)'], errors='coerce'
)

# 3. Collapse B/C → "B+C", drop D
dep_IPI_full['Level 1 Index'] = (
    dep_IPI_full['Level 1 Index']
    .astype(str).str.strip()
    .replace({'B': 'B+C', 'C': 'B+C', 'D': np.nan})
)
dep_IPI_full = dep_IPI_full.dropna(subset=['Level 1 Index'])

# 4. Aggregate (sum) over Country, Time, Level 1 Index
dep_IPI_agg = (
    dep_IPI_full
    .groupby(['Country', 'Time', 'Level 1 Index'], as_index=False)
    ['Indprod Index Value (I21)']
    .sum()
)

# 5. Convert Time to Period and sort
dep_IPI_agg['Time_period'] = pd.to_datetime(dep_IPI_agg['Time']).dt.to_period('M')
dep_IPI_agg = dep_IPI_agg.sort_values(['Country', 'Level 1 Index', 'Time_period'])

# ------------------------------------------------------------------
# 6. COMPUTE DIFF + LAG ON FULL DATA (KEY STEP!)
# ------------------------------------------------------------------
dep_IPI_agg['diff_IPI'] = (
    dep_IPI_agg
    .groupby(['Country', 'Level 1 Index'])['Indprod Index Value (I21)']
    .diff()
)

dep_IPI_agg['lagged_diff_IPI'] = (
    dep_IPI_agg
    .groupby(['Country', 'Level 1 Index'])['diff_IPI']
    .shift(1)
)

# ------------------------------------------------------------------
# 7. NOW FILTER TO YOUR ANALYSIS WINDOW
# ------------------------------------------------------------------
mask = (
    (dep_IPI_agg['Time_period'] >= pd.Period(start_date, 'M')) &
    (dep_IPI_agg['Time_period'] <= pd.Period(end_date, 'M'))
)
dep_IPI_final = dep_IPI_agg.loc[mask].copy()

# ------------------------------------------------------------------
# 8. Final cleanup & save
# ------------------------------------------------------------------
final_cols = [
    'Country', 'Level 1 Index', 'Time',
    'Indprod Index Value (I21)',  # current level
    'diff_IPI',                   # ΔIPI_t
    'lagged_diff_IPI'             # ΔIPI_{t-1} ← FULLY POPULATED
]
dep_IPI_final = dep_IPI_final[final_cols]

Path('data/dependent_variable').mkdir(parents=True, exist_ok=True)
dep_IPI_final.to_csv(
    'data/dependent_variable/IndustrialProductionIndex_df.csv',
    index=False
)

print("Saved: IndustrialProductionIndex_df.csv")
print(f"  → Analysis window: {start_date} to {end_date}")
print(f"  → Valid lagged differences: {dep_IPI_final['lagged_diff_IPI'].notna().sum():,}")
print(f"  → First month in window has lag: {dep_IPI_final.iloc[0]['lagged_diff_IPI'] is not pd.NA}")

# Stocks Dependent
stock_period = pd.to_datetime(dep_Stocks['Time'], errors='coerce').dt.to_period('M')
stock_mask = (stock_period >= pd.Period(start_date, 'M')) & (stock_period <= pd.Period(end_date, 'M'))
stock_cols = ['Country', 'Stock Index', 'Time', 'Log Monthly Return', 'Volume']
dep_Stocks_filtered = dep_Stocks.loc[stock_mask, stock_cols].copy()
dep_Stocks_filtered = dep_Stocks_filtered.sort_values(['Country', 'Stock Index', 'Time']).reset_index(drop=True)
dep_Stocks_filtered.to_csv('data/dependent_variable/StockIndex_df.csv', index=False)

# Tariff(i,t) Independent

# 1. Round publication dates to month (same as before)
dt = pd.to_datetime(exposure_df['Publication_Date'], errors='coerce')
month_start = dt.dt.to_period('M').dt.start_time
next_month_start = (dt.dt.to_period('M') + 1).dt.start_time
delta_to_start = (dt - month_start).dt.days
delta_to_next = (next_month_start - dt).dt.days
rounded_month_start = pd.to_datetime(
    np.where(delta_to_start <= delta_to_next, month_start, next_month_start)
)

exposure_out = exposure_df.copy()
exposure_out['Time_dt'] = rounded_month_start
exposure_out['Time'] = exposure_out['Time_dt'].dt.strftime('%Y-%m')
exposure_out = exposure_out[['Country', 'Time', 'Exposure']].copy()

# 2. Define full panel: all countries × all months (2024-10 to 2025-08)
#     → 2024-10 needed for lag of 2024-11
all_countries = sorted(exposure_out['Country'].unique())
all_months = pd.date_range('2024-10', '2025-08', freq='MS').strftime('%Y-%m').tolist()

full_index = pd.MultiIndex.from_product([all_countries, all_months], names=['Country', 'Time'])
full_panel = pd.DataFrame(index=full_index).reset_index()

# 3. Merge observed shocks, fill missing with 0
exposure_full = full_panel.merge(exposure_out, on=['Country', 'Time'], how='left')
exposure_full['Exposure'] = exposure_full['Exposure'].fillna(0)

# 4. Sort and save
exposure_full = exposure_full.sort_values(['Country', 'Time']).reset_index(drop=True)
Path('data/independent_variable').mkdir(parents=True, exist_ok=True)
exposure_full.to_csv('data/independent_variable/CountryTariffExposure_df.csv', index=False)

# Openess(i,t) Independent
time_raw = openness_df['Time'].astype(str).str.strip()
year_num = pd.to_numeric(time_raw, errors='coerce')
year_dt = pd.to_datetime(time_raw, errors='coerce').dt.year
year = year_num.fillna(year_dt)
mask = year.isin([2024, 2025])
openness_out = openness_df.loc[mask].copy()
openness_out['Time'] = year.loc[mask].astype('Int64').astype(str)
openness_out = openness_out.sort_values(['Country', 'Time']).reset_index(drop=True)
openness_out.to_csv('data/transition_variable/TradeOpennessAnnual_df.csv', index=False)




# wb_openess(i,t)
# Step 1: Clean and convert Time to numeric year
wb_time_raw = wb_openness_df['Time'].astype(str).str.strip()
wb_year_num = pd.to_numeric(wb_time_raw, errors='coerce')
wb_year_dt = pd.to_datetime(wb_time_raw, errors='coerce').dt.year
wb_year = wb_year_num.fillna(wb_year_dt)
# Assign cleaned year back
wb_openness_df['Time'] = wb_year
# Step 2: Create 2025 entries with 0 openness for all countries
countries = wb_openness_df['Country'].unique()
year_2025_df = pd.DataFrame({
    'Country': countries,
    'Time': 2025,
    'Global Trade Openness (%GDP)': 0,
    'Global Trade Openness-Lag1': pd.NA  # Will be filled after lagging
})
# Append 2025 data
wb_openness_extended = pd.concat([wb_openness_df, year_2025_df], ignore_index=True)
# Step 3: Sort by Country and Time to prepare for lagging
wb_openness_extended = wb_openness_extended.sort_values(['Country', 'Time']).reset_index(drop=True)
# Step 4: Create lag (Global Trade Openness-Lag1) — shift within each country
wb_openness_extended['Global Trade Openness-Lag1'] = (
    wb_openness_extended.groupby('Country')['Global Trade Openness (%GDP)']
    .shift(1)
)
# Step 5: Filter only 2023 and 2024
wb_mask = wb_openness_extended['Time'].isin([2024, 2025])
wb_openness_out = wb_openness_extended.loc[wb_mask].copy()
# Step 6: Format Time as string (Int64 -> str)
wb_openness_out['Time'] = wb_openness_out['Time'].astype('Int64').astype(str)
# Step 7: Final sort and save
wb_openness_out = wb_openness_out.sort_values(['Country', 'Time']).reset_index(drop=True)
wb_openness_out.to_csv('data/transition_variable/WBTradeOpennessAnnual_df.csv', index=False)


# ================================
# Transition Variable + LAG (CORRECT & WORKING)
# ================================
cols = [
    'STRUCTURE','STRUCTURE_ID','STRUCTURE_NAME','freq','Frequency',
    'Country','REPORTER','partner','PARTNER','product','PRODUCT',
    'flow','FLOW','indicators','INDICATORS','Time','TIME_PERIOD',
    'OBS_VALUE','Observation Value'
]

df_full = transition_df[cols].copy()

# ------------------------------------------------------------------
# 1. Prepare time & numeric columns
# ------------------------------------------------------------------
df_full['Time_period'] = pd.to_datetime(
    df_full['Time'], errors='coerce'
).dt.to_period('M')

df_full['OBS_VALUE'] = pd.to_numeric(df_full['OBS_VALUE'], errors='coerce')

df_full = df_full.sort_values(['Country', 'Time_period']).reset_index(drop=True)

# ------------------------------------------------------------------
# 2. LAG (on original level)
# ------------------------------------------------------------------
df_full['OBS_VALUE_Lagged1'] = df_full.groupby('Country')['OBS_VALUE'].shift(1)

# ------------------------------------------------------------------
# 3. Filter by date window
# ------------------------------------------------------------------
mask = (
    (df_full['Time_period'] >= pd.Period(start_date, 'M')) &
    (df_full['Time_period'] <= pd.Period(end_date, 'M'))
)
transition_df_filtered = df_full.loc[mask].copy()
transition_df_filtered = transition_df_filtered.drop(columns=['Time_period'])
transition_df_filtered = transition_df_filtered.sort_values(['Country', 'Time']).reset_index(drop=True)

# ------------------------------------------------------------------
# 4. ---- BOTH CURRENT & LAGGED VALUES AS PROPORTIONS (FIXED) ----
# ------------------------------------------------------------------

# Step 1: Compute total export PER MONTH (one value per month)
monthly_totals = transition_df_filtered.groupby('Time')['OBS_VALUE'].sum()

# Step 2: Map current month total to each row
transition_df_filtered['monthly_total_current'] = transition_df_filtered['Time'].map(monthly_totals)

# Step 3: Create a shifted version of monthly totals (previous month total)
monthly_totals_lagged = monthly_totals.shift(1)

# Step 4: Map lagged month total to each row
transition_df_filtered['monthly_total_lagged'] = transition_df_filtered['Time'].map(monthly_totals_lagged)

# Step 5: Convert to proportions
transition_df_filtered['OBS_VALUE'] = (
    transition_df_filtered['OBS_VALUE'] / transition_df_filtered['monthly_total_current']
)

# Lagged proportion: lagged level / previous month's total
transition_df_filtered['OBS_VALUE_Lagged1'] = (
    transition_df_filtered['OBS_VALUE_Lagged1'] / transition_df_filtered['monthly_total_lagged']
)

# OPTIONAL: Clean up helper columns
transition_df_filtered = transition_df_filtered.drop(columns=['monthly_total_current', 'monthly_total_lagged'])

# OPTIONAL: Convert to percentages (0–100%)
# transition_df_filtered['OBS_VALUE'] *= 100
# transition_df_filtered['OBS_VALUE_Lagged1'] *= 100

# ------------------------------------------------------------------
# 5. Save
# ------------------------------------------------------------------
Path('data/transition_variable').mkdir(parents=True, exist_ok=True)
transition_df_filtered.to_csv(
    'data/transition_variable/EU_partner_index_df.csv', index=False
)

print(f"Lag added using pre-period data!")
print(f"Valid lags: {transition_df_filtered['OBS_VALUE_Lagged1'].notna().sum():,}")
print("File saved: EU_partner_index_df.csv  (both OBS_VALUE and OBS_VALUE_Lagged1 are **proportions** of their respective month's total)")

# ================================
# Controls (i,t) – LAGGED HICP & LAGGED ΔUnemployment
# ================================

controls_cols = [
    'Country',
    'Time',
    'GDP (Million USD)',
    'HICP (%, annual rate of change)',
    'Unemployment Rate (%pop in LF)'
]

controls_full = controls_df[controls_cols].copy()

# 1. Convert Time to datetime (same as before)
controls_full['Time_dt'] = pd.to_datetime(controls_full['Time'], errors='coerce')
controls_full = controls_full.sort_values(['Country', 'Time_dt'])

# ------------------------------------------------------------------
# 2. FIRST DIFFERENCE UNEMPLOYMENT (still needed for the lag)
# ------------------------------------------------------------------
controls_full['ΔUnemployment'] = (
    controls_full.groupby('Country')['Unemployment Rate (%pop in LF)'].diff()
)

# ------------------------------------------------------------------
# 3. CREATE LAG-1 FOR BOTH VARIABLES (within country)
# ------------------------------------------------------------------
controls_full['HICP_lag1'] = (
    controls_full.groupby('Country')['HICP (%, annual rate of change)'].shift(1)
)

controls_full['Unemployment_lag1'] = (
    controls_full.groupby('Country')['ΔUnemployment'].shift(1)
)

# ------------------------------------------------------------------
# 4. FILTER TO ANALYSIS WINDOW (2024-11 → 2025-08)
# ------------------------------------------------------------------
controls_period = controls_full['Time_dt'].dt.to_period('M')
controls_mask = (
    (controls_period >= pd.Period(start_date, 'M')) &
    (controls_period <= pd.Period(end_date, 'M'))
)

controls_out = controls_full.loc[controls_mask, [
    'Country',
    'Time_dt',
    'GDP (Million USD)',
    'HICP_lag1',               # <-- lagged HICP
    'Unemployment_lag1'       # <-- lagged ΔUnemployment
]].copy()

# ------------------------------------------------------------------
# 5. FORMAT TIME COLUMN AND CLEAN UP
# ------------------------------------------------------------------
controls_out['Time'] = controls_out['Time_dt'].dt.strftime('%Y-%m')
controls_out = controls_out.drop(columns=['Time_dt'])

# final column order for the CSV
controls_out = controls_out[[
    'Country',
    'Time',
    'GDP (Million USD)',
    'HICP_lag1',
    'Unemployment_lag1'
]]

controls_out = controls_out.sort_values(['Country', 'Time']).reset_index(drop=True)

# ------------------------------------------------------------------
# 6. SAVE
# ------------------------------------------------------------------
Path('data/control_variable').mkdir(parents=True, exist_ok=True)
controls_out.to_csv('data/control_variable/CountryControls_df.csv', index=False)

print("\nControls saved with:")
print("  • HICP_lag1  = HICP(t-1)")
print("  • ΔUnemployment_lag1 = ΔUnemployment(t-1)")
print(f"  • Window: {start_date} → {end_date}")
print(f"  • Valid HICP lags: {controls_out['HICP_lag1'].notna().sum():,}")
print(f"  • Valid ΔUnemp lags: {controls_out['Unemployment_lag1'].notna().sum():,}\n")



Dropped 924 non-EU rows → 25,870 EU rows kept
Dropped 6 non-EU rows → 156 EU rows kept
Dropped 1,707 non-EU rows → 11,035 EU rows kept
Dropped 316 non-EU rows → 521 EU rows kept

Kept only 27 EU countries: ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']

Saved: IndustrialProductionIndex_df.csv
  → Analysis window: 2024-11 to 2025-08
  → Valid lagged differences: 260
  → First month in window has lag: True
Lag added using pre-period data!
Valid lags: 224
File saved: EU_partner_index_df.csv  (both OBS_VALUE and OBS_VALUE_Lagged1 are **proportions** of their respective month's total)

Controls saved with:
  • HICP_lag1  = HICP(t-1)
  • ΔUnemployment_lag1 = ΔUnemployment(t-1)
  • Window: 2024-11 → 2025-08
  • Valid HICP lags: 260
  • Valid ΔUnemp lags: 260



## 4) Baseline Simple OLS regressions

We estimate the **short-run effect** of U.S. tariff exposure on monthly macroeconomic outcomes (Industrial Production Index (B+C), Stocks Index Return, FX change).
The model includes **country** and **month fixed effects** to control for unobserved, time-invariant country characteristics and global shocks.

$$
y_{i,t} = \mu_i + \lambda_t
+ \alpha_0\,\text{Exposure}_{i,t}
+ \alpha_1\,\text{Exposure}_{i,t-1}
+ \Gamma' Z_{i,t}
+ \varepsilon_{i,t}
$$

**Where:**

- $y_{i,t}$ — Outcome variable (Industrial Production Index (B+C) YoY for country *i* in month *t*, Stocks Index for country *i* in month *t*
- $\text{Exposure}_{i,t}$ — Country *i*’s effective tariff exposure in month *t*
- $\text{Exposure}_{i,t-1}$ — One-month lag of tariff exposure (captures delayed reaction)
- $Z_{i,t}$ — Vector of monthly country-specific control variables (HICP YoY%, Δ unemployment)
- $\mu_i$ — Country fixed effects (absorbing structural country heterogeneity)
- $\lambda_t$ — Month fixed effects (absorbing global macro shocks)
- $\varepsilon_{i,t}$ — Idiosyncratic error term, clustered at the country level

---

### **Estimation Details**

- **Estimator:** OLS with two-way (country and month) fixed effects
- **Frequency:** Monthly (2024-10 → 2025-08)
- **Standard Errors:** Clustered by country
- **Sample:** EU countries only, using *effective* tariff exposure measure
- **Controls:** Domestic macro variables (HICP YoY%, Δ unemployment)

---

### **Parameters of Interest**

- $\alpha_0$ — Contemporaneous effect of tariff exposure
- $\alpha_1$ — One-month-lagged effect
- $\alpha_0 + \alpha_1$ — Cumulative 0–1-month effect (interpreted as the total short-run impact)


In [3]:

BASE = Path.cwd() / "data"

ipi      = pd.read_csv(BASE / "dependent_variable" / "IndustrialProductionIndex_df.csv")
stocks   = pd.read_csv(BASE / "dependent_variable" / "StockIndex_df.csv")
exposure = pd.read_csv(BASE / "independent_variable" / "CountryTariffExposure_df.csv")
controls = pd.read_csv(BASE / "control_variable" / "CountryControls_df.csv")


# Convert Time → Period[M] (keeps month arithmetic)

for df in (ipi, stocks, exposure, controls):
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce").dt.to_period("M")

# Choose Y
# ---- Industrial Production (B+C) YoY ----
df_dep = ipi.rename(columns={"diff_IPI": "y"})
# ---- Uncomment to run on Stock log-returns instead ----
#df_dep = stocks.rename(columns={"Log Monthly Return": "y"})
#


exp = exposure.rename(columns={"Exposure": "Exposure_t"})
exp = exp.sort_values(["Country", "Time"])
exp["Exposure_t1"] = exp.groupby("Country")["Exposure_t"].shift(1)
exp = exp.dropna(subset=["Exposure_t1"])

df = (
    df_dep[["Country", "Time", "y"]]
    .merge(exp[["Country", "Time", "Exposure_t", "Exposure_t1"]],
           on=["Country", "Time"], how="inner")
    .merge(controls[["Country", "Time",
                     "HICP_lag1",
                     "Unemployment_lag1"]],
           on=["Country", "Time"], how="inner")
)

df = df[(df["Time"] >= pd.Period("2024-11", "M")) &
        (df["Time"] <= pd.Period("2025-08", "M"))]

df["Time_dt"] = df["Time"].dt.start_time          # first day of month → datetime64
df = df.set_index(["Country", "Time_dt"])
df = df.drop(columns=["Time"])                    # optional cleanup

print("Months in final panel:", sorted(df.index.get_level_values(1).unique()))
print("Obs per country (min/avg/max):",
      df.groupby(level=0).size().min(),
      df.groupby(level=0).size().mean().round(1),
      df.groupby(level=0).size().max())


# Run two-way FE regression (cluster by country)

exog_vars = [
    "Exposure_t",
    "Exposure_t1",
    "HICP_lag1",
    "Unemployment_lag1"
]

mod = PanelOLS(
    dependent=df["y"],
    exog=df[exog_vars],
    entity_effects=True,   # μ_i
    time_effects=True,     # λ_t
)

res = mod.fit(cov_type="clustered", cluster_entity=True)
print(res)

Months in final panel: [Timestamp('2024-11-01 00:00:00'), Timestamp('2024-12-01 00:00:00'), Timestamp('2025-01-01 00:00:00'), Timestamp('2025-02-01 00:00:00'), Timestamp('2025-03-01 00:00:00'), Timestamp('2025-04-01 00:00:00'), Timestamp('2025-05-01 00:00:00'), Timestamp('2025-06-01 00:00:00'), Timestamp('2025-07-01 00:00:00'), Timestamp('2025-08-01 00:00:00')]
Obs per country (min/avg/max): 10 10.0 10
                          PanelOLS Estimation Summary                           
Dep. Variable:                      y   R-squared:                        0.0369
Estimator:                   PanelOLS   R-squared (Between):             -4.7928
No. Observations:                 260   R-squared (Within):              -0.0113
Date:                Sat, Nov 08 2025   R-squared (Overall):             -0.2654
Time:                        13:18:56   Log-likelihood                   -1088.7
Cov. Estimator:             Clustered                                           
                           

## 5) Extended Specification — Linear Heterogeneity with Trade Openness Interaction

To allow the impact of tariff exposure to vary across countries with different levels of trade openness,
we extend the baseline specification by interacting exposure with lagged openness.
This captures whether **more open economies** react differently to tariff shocks.

$$
y_{i,t} = \mu_i + \lambda_t
+ \alpha_0\,\text{Exposure}_{i,t}
+ \alpha_1\,\text{Exposure}_{i,t-1}
+ \beta_0\,(\text{Exposure}_{i,t} \times \text{Openness}_{i,t-1}^{US})
+ \beta_1\,(\text{Exposure}_{i,t-1} \times \text{Openness}_{i,t-1}^{US})
+ \Gamma' Z_{i,t}
+ \varepsilon_{i,t}
$$

**Where:**

- $y_{i,t}$ — Outcome variable (e.g. Industrial Production YoY for country *i* in month *t*)
- $\text{Exposure}_{i,t}$ — Effective tariff exposure in month *t*
- $\text{Exposure}_{i,t-1}$ — One-month lag of exposure
- $\text{Openness}_{i,t-1}^{US}$ — Lagged trade openness index relative to the US (annual, mapped to months)
- $(\text{Exposure}_{i,t} \times \text{Openness}_{i,t-1}^{US})$ — Interaction term capturing heterogeneous exposure effects
- $Z_{i,t}$ — Monthly domestic controls (HICP YoY, Δ unemployment)
- $\mu_i$ — Country fixed effects
- $\lambda_t$ — Month fixed effects
- $\varepsilon_{i,t}$ — Error term clustered by country


### **Interpretation**

- $\alpha_0$, $\alpha_1$ — Baseline (average) effects of exposure at zero openness
- $\beta_0$, $\beta_1$ — Marginal change in the exposure effect as openness increases
- **Marginal effect of exposure:**
  $$
  \frac{\partial y_{i,t}}{\partial \text{Exposure}_{i,t}}
  = \alpha_0 + \beta_0\,\text{Openness}_{i,t-1}^{US}
  $$
  and
  $$
  \frac{\partial y_{i,t}}{\partial \text{Exposure}_{i,t-1}}
  = \alpha_1 + \beta_1\,\text{Openness}_{i,t-1}^{US}
  $$
- Report **cumulative marginal effect** $(\alpha_0+\alpha_1) + (\beta_0+\beta_1)\text{Openness}_{i,t-1}^{US}$ evaluated at low, median, and high openness levels.

---

### **Estimation Details**

- **Estimator:** OLS with two-way (country and month) fixed effects
- **Frequency:** Monthly (2024-10 → 2025-09)
- **Standard Errors:** Clustered by country
- **Sample:** EU countries only
- **Controls:** Domestic macro variables (HICP YoY, unemployment)

---

### **Objective**

This model tests whether the **sensitivity to U.S. tariff shocks** depends on how open a country's economy is to the US.
A positive $\beta_0$ or $\beta_1$ implies that **more open economies are more affected** by tariff changes,
while a negative coefficient indicates **buffering or diversification effects** from openness.


In [4]:
# Traditional Trade Openness
BASE = Path.cwd() / "data"

ipi        = pd.read_csv(BASE / "dependent_variable" / "IndustrialProductionIndex_df.csv")
stocks     = pd.read_csv(BASE / "dependent_variable" / "StockIndex_df.csv")
exposure   = pd.read_csv(BASE / "independent_variable" / "CountryTariffExposure_df.csv")
controls   = pd.read_csv(BASE / "control_variable" / "CountryControls_df.csv")
openness   = pd.read_csv(BASE / "transition_variable" / "TradeOpennessAnnual_df.csv")

for df in (ipi, stocks, exposure, controls):
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce").dt.to_period("M")

openness = openness[["Country", "Time", "Trade_Openness_pct_GDP"]].copy()
openness.rename(columns={"Trade_Openness_pct_GDP": "Openness_annual"}, inplace=True)
openness["Year"] = pd.to_datetime(openness["Time"].astype(str), errors="coerce").dt.year
openness = openness.drop(columns=["Time"])

monthly_open = []
for year in [2024, 2025]:
    months = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq="MS")
    temp   = pd.DataFrame({"Time_dt": months})
    temp["Time"] = temp["Time_dt"].dt.to_period("M")
    temp = temp.merge(openness[openness["Year"] == year], how="cross")
    monthly_open.append(temp)

openness_monthly = pd.concat(monthly_open, ignore_index=True)
openness_monthly = openness_monthly[["Country", "Time", "Openness_annual"]]
openness_monthly = openness_monthly.rename(columns={"Openness_annual": "Openness"})

#df_dep = stocks.rename(columns={"Log Monthly Return": "y"})
df_dep = ipi.rename(columns={"diff_IPI": "y"})

exp = exposure.rename(columns={"Exposure": "Exposure_t"})
exp = exp.sort_values(["Country", "Time"])
exp["Exposure_t1"] = exp.groupby("Country")["Exposure_t"].shift(1)

exp = exp.merge(openness_monthly, on=["Country", "Time"], how="left")
exp["Openness_t1"] = exp.groupby("Country")["Openness"].shift(1)

exp = exp.dropna(subset=["Exposure_t1", "Openness_t1"])

exp["Exp_t_x_Open_t1"]   = exp["Exposure_t"]   * exp["Openness_t1"]
exp["Exp_t1_x_Open_t1"]  = exp["Exposure_t1"]  * exp["Openness_t1"]

df = (
    df_dep[["Country", "Time", "y"]]
    .merge(exp[["Country", "Time",
                "Exposure_t", "Exposure_t1",
                "Exp_t_x_Open_t1", "Exp_t1_x_Open_t1",
                "Openness_t1"]],
           on=["Country", "Time"], how="inner")
    .merge(controls[["Country", "Time",
                     "HICP_lag1",
                     "Unemployment_lag1"]],
           on=["Country", "Time"], how="inner")
)

df = df[(df["Time"] >= pd.Period("2024-11", "M")) &
        (df["Time"] <= pd.Period("2025-08", "M"))]

df["Time_dt"] = df["Time"].dt.start_time
df = df.set_index(["Country", "Time_dt"])
df = df.drop(columns=["Time"])

exog_vars = [
    "Exposure_t",
    "Exposure_t1",
    "Exp_t_x_Open_t1",
    "Exp_t1_x_Open_t1",
    "Openness_t1",
    "HICP_lag1",
    "Unemployment_lag1"
]

mod = PanelOLS(
    dependent=df["y"],
    exog=df[exog_vars],
    entity_effects=True,
    time_effects=True,
)

res = mod.fit(cov_type="clustered", cluster_entity=True)
print(res)

print("\n=== MARGINAL EFFECTS OF TARIFF EXPOSURE ===")
open_levels = {
    "Low":     df["Openness_t1"].quantile(0.25),
    "Median":  df["Openness_t1"].median(),
    "High":    df["Openness_t1"].quantile(0.75)
}

for label, q in open_levels.items():
    me_t = res.params["Exposure_t"] + res.params["Exp_t_x_Open_t1"] * q
    var_t = (res.cov.loc["Exposure_t", "Exposure_t"] +
             2 * q * res.cov.loc["Exposure_t", "Exp_t_x_Open_t1"] +
             q**2 * res.cov.loc["Exp_t_x_Open_t1", "Exp_t_x_Open_t1"])
    se_t = np.sqrt(var_t)

    me_t1 = res.params["Exposure_t1"] + res.params["Exp_t1_x_Open_t1"] * q
    var_t1 = (res.cov.loc["Exposure_t1", "Exposure_t1"] +
              2 * q * res.cov.loc["Exposure_t1", "Exp_t1_x_Open_t1"] +
              q**2 * res.cov.loc["Exp_t1_x_Open_t1", "Exp_t1_x_Open_t1"])
    se_t1 = np.sqrt(var_t1)

    cov_cross = (res.cov.loc["Exposure_t", "Exposure_t1"] +
                 q * (res.cov.loc["Exposure_t", "Exp_t1_x_Open_t1"] +
                      res.cov.loc["Exp_t_x_Open_t1", "Exposure_t1"]) +
                 q**2 * res.cov.loc["Exp_t_x_Open_t1", "Exp_t1_x_Open_t1"])
    var_cum = var_t + var_t1 + 2 * cov_cross
    se_cum = np.sqrt(var_cum)
    me_cum = me_t + me_t1

    print(f"\n{label} Openness ({q:.4f}):")
    print(f"  Contemporaneous : {me_t: .5f} (se = {se_t: .5f})")
    print(f"  Lagged          : {me_t1:.5f} (se = {se_t1:.5f})")
    print(f"  Cumulative      : {me_cum:.5f} (se = {se_cum:.5f})")


                          PanelOLS Estimation Summary                           
Dep. Variable:                      y   R-squared:                        0.0436
Estimator:                   PanelOLS   R-squared (Between):             -3.7374
No. Observations:                 260   R-squared (Within):              -0.0004
Date:                Sat, Nov 08 2025   R-squared (Overall):             -0.1990
Time:                        13:21:59   Log-likelihood                   -1087.8
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.4195
Entities:                          26   P-value                           0.1986
Avg Obs:                      10.0000   Distribution:                   F(7,218)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             4.3066
                            

## 6) Extended Specification — PSTR Model with Smooth Transition by Trade Openness

We now allow the effect of tariff exposure to vary **smoothly** with the level of trade openness with the US.
Instead of assuming a linear interaction, we introduce a **logistic transition function** that captures gradual changes in the impact of exposure as openness increases.

The logistic transition function is defined as:

$$
G\!\left(\text{Openness}_{i,t-1}^{US};\gamma,c\right)
= \frac{1}{1 + \exp\!\big[-\gamma\big(\text{Openness}_{i,t-1}^{US} - c\big)\big]}.
$$

The corresponding PSTR regression is:

$$
\begin{aligned}
y_{i,t} &= \mu_i + \lambda_t
+ \alpha_0\,\text{Exposure}_{i,t}
+ \alpha_1\,\text{Exposure}_{i,t-1} \\
&\quad + \big(\beta_0\,\text{Exposure}_{i,t} + \beta_1\,\text{Exposure}_{i,t-1}\big)
\,G\!\left(\text{Openness}_{i,t-1}^{US};\gamma,c\right)
+ \Gamma' Z_{i,t}
+ \varepsilon_{i,t}
\end{aligned}
$$

**Where:**

- $y_{i,t}$ — Outcome variable (e.g. Industrial Production YoY for country *i* in month *t*)
- $\text{Exposure}_{i,t}$ — Effective tariff exposure in month *t*
- $\text{Exposure}_{i,t-1}$ — One-month lag of exposure
- $\text{Openness}_{i,t-1}^{US}$ — Lagged trade openness index (annual, mapped to months)
- $G(\text{Openness}_{i,t-1}^{US};\gamma,c)$ — Logistic transition function with:
  - $c$: threshold (location) where $G=0.5$
  - $\gamma$: smoothness parameter controlling how sharp the transition is
- $Z_{i,t}$ — Monthly domestic controls (HICP YoY, Δ unemployment)
- $\mu_i$ — Country fixed effects
- $\lambda_t$ — Month fixed effects
- $\varepsilon_{i,t}$ — Error term clustered by country

---

### **Interpretation**

- $\alpha_0$, $\alpha_1$ — Baseline exposure effects when openness is low ($G \approx 0$)
- $\beta_0$, $\beta_1$ — Incremental effects as openness increases ($G \to 1$)
- **Marginal effects of exposure:**
  $$
  \frac{\partial y_{i,t}}{\partial \text{Exposure}_{i,t}}
  = \alpha_0 + \beta_0\,G(\text{Openness}_{i,t-1}^{US};\gamma,c),
  $$
  $$
  \frac{\partial y_{i,t}}{\partial \text{Exposure}_{i,t-1}}
  = \alpha_1 + \beta_1\,G(\text{Openness}_{i,t-1}^{US};\gamma,c).
  $$
- **Cumulative short-run effect (0–1 month):**
  $$
  (\alpha_0 + \alpha_1) + (\beta_0 + \beta_1)\,G(\text{Openness}_{i,t-1}^{US};\gamma,c).
  $$
- When $\gamma$ is large, $G(\cdot)$ approximates a sharp threshold model; when small, the transition is smooth.

---

### **Estimation Details**

- **Estimator:** Nonlinear least squares with two-way (country and month) fixed effects
- **Frequency:** Monthly (2024-10 → 2025-09)
- **Standard Errors:** Clustered by country
- **Sample:** EU countries only
- **Controls:** Domestic macro variables (HICP YoY, unemployment)
- **Pre-processing:** Standardize openness before estimation; report the threshold $c$ in original units after transformation

---

### **Objective**

This model identifies whether the **effect of U.S. tariff shocks on economic outcomes** depends on how open each country is to trade with the US.
The logistic transition function captures a **nonlinear response** — for example, more open economies may only become significantly affected **after** crossing a critical openness threshold.


In [5]:
# Traditional Trade Openness 
BASE = Path.cwd() / "data"

ipi        = pd.read_csv(BASE / "dependent_variable" / "IndustrialProductionIndex_df.csv")
stocks     = pd.read_csv(BASE / "dependent_variable" / "StockIndex_df.csv")
exposure   = pd.read_csv(BASE / "independent_variable" / "CountryTariffExposure_df.csv")
controls   = pd.read_csv(BASE / "control_variable" / "CountryControls_df.csv")
openness   = pd.read_csv(BASE / "transition_variable" / "TradeOpennessAnnual_df.csv")

for df in (ipi, stocks, exposure, controls):
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce").dt.to_period("M")

openness = openness[["Country", "Time", "Openness_Lag1"]].copy()
openness.rename(columns={"Openness_Lag1": "Openness_annual"}, inplace=True)
openness["Year"] = pd.to_datetime(openness["Time"].astype(str), errors="coerce").dt.year
openness = openness.drop(columns=["Time"])

monthly_open = []
for year in [2024, 2025]:
    months = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq="MS")
    tmp = pd.DataFrame({"Time_dt": months})
    tmp["Time"] = tmp["Time_dt"].dt.to_period("M")
    tmp = tmp.merge(openness[openness["Year"] == year][["Country", "Openness_annual"]], how="cross")
    monthly_open.append(tmp)

openness_monthly = pd.concat(monthly_open, ignore_index=True)
openness_monthly = openness_monthly[["Country", "Time", "Openness_annual"]]
openness_monthly = openness_monthly.rename(columns={"Openness_annual": "Openness"})

#df_dep = stocks.rename(columns={"Log Monthly Return": "y"})
df_dep = ipi.rename(columns={"diff_IPI": "y"})

exp = exposure.rename(columns={"Exposure": "Exposure_t"}).sort_values(["Country", "Time"])
exp["Exposure_t1"] = exp.groupby("Country")["Exposure_t"].shift(1)

exp = exp.merge(openness_monthly, on=["Country", "Time"], how="left")
exp["Openness_t1"] = exp.groupby("Country")["Openness"].shift(1)

exp = exp.dropna(subset=["Exposure_t1", "Openness_t1"])

df = (
    df_dep[["Country", "Time", "y"]]
    .merge(exp[["Country", "Time", "Exposure_t", "Exposure_t1", "Openness_t1"]],
           on=["Country", "Time"], how="inner")
    .merge(controls[["Country", "Time",
                     "HICP_lag1",
                     "Unemployment_lag1"]],
           on=["Country", "Time"], how="inner")
)

df = df[(df["Time"] >= pd.Period("2024-11", "M")) &
        (df["Time"] <= pd.Period("2025-08", "M"))].copy()

df["Time_dt"] = df["Time"].dt.start_time
df = df.set_index(["Country", "Time_dt"]).sort_index()
df.drop(columns=["Time"], inplace=True)

open_mean = df["Openness_t1"].mean()
open_std  = df["Openness_t1"].std(ddof=0)
df["Open_std"] = (df["Openness_t1"] - open_mean) / open_std

df = df.rename(columns={
    "HICP_lag1": "HICP",
    "Unemployment_lag1": "Unemp"
})

def twoway_within_transform(panel_df, cols):
    """
    Double-demean columns (entity & time FE).
    Returns a new DataFrame with *_wt (within-transformed) columns.
    """
    out = panel_df.copy()
    ent_means = out.groupby(level=0)[cols].transform("mean")
    time_means = out.groupby(level=1)[cols].transform("mean")
    overall_means = out[cols].mean()
    wt = out[cols] - ent_means - time_means + overall_means
    wt.columns = [c + "_wt" for c in cols]
    return wt

base_cols = ["y", "Exposure_t", "Exposure_t1", "HICP", "Unemp", "Open_std"]
wt = twoway_within_transform(df, base_cols)
for c in wt.columns:
    df[c] = wt[c]

def G_logistic(z, gamma, c):
    return 1.0 / (1.0 + np.exp(-gamma * (z - c)))

def build_X_within(gamma, c, work_df):
    """
    Build within-transformed X for given (γ,c):
      [ Exp_t_wt, Exp_t1_wt, (Exp_t*G)_wt, (Exp_t1*G)_wt, HICP_wt, Unemp_wt ]
    """
    z = work_df["Open_std_wt"].to_numpy()
    G = G_logistic(z, gamma, c)
    X = np.column_stack([
        work_df["Exposure_t_wt"].to_numpy(),
        work_df["Exposure_t1_wt"].to_numpy(),
        work_df["Exposure_t_wt"].to_numpy()  * G,
        work_df["Exposure_t1_wt"].to_numpy() * G,
        work_df["HICP_wt"].to_numpy(),
        work_df["Unemp_wt"].to_numpy(),
    ])
    return X

y_wt = df["y_wt"].to_numpy()

def ssr_objective(theta):
    gamma, c = theta
    if gamma <= 0:
        return 1e12 + (abs(gamma) + 1.0) * 1e12
    X = build_X_within(gamma, c, df)
    beta_hat, *_ = np.linalg.lstsq(X, y_wt, rcond=None)
    resid = y_wt - X @ beta_hat
    return float(resid @ resid)

theta0 = np.array([1.0, 0.0])
bounds = [(1e-3, 100.0), (-3.0, 3.0)]

opt = minimize(ssr_objective, theta0, method="L-BFGS-B", bounds=bounds)
gamma_hat, c_hat_std = opt.x
print("\n=== PSTR (NLS) transition estimates ===")
print(f"  gamma (smoothness): {gamma_hat: .4f}")
print(f"  c (threshold, standardized): {c_hat_std: .4f}")
c_hat_orig = open_mean + open_std * c_hat_std
print(f"  c (threshold, ORIGINAL openness units): {c_hat_orig: .4f}")

G_hat = G_logistic(df["Open_std"].to_numpy(), gamma_hat, c_hat_std)
df["G_hat"] = G_hat

df["Exp_t_G"]   = df["Exposure_t"]  * df["G_hat"]
df["Exp_t1_G"]  = df["Exposure_t1"] * df["G_hat"]

exog_cols = ["Exposure_t", "Exposure_t1", "Exp_t_G", "Exp_t1_G", "HICP", "Unemp"]

mod = PanelOLS(
    dependent=df["y"],
    exog=df[exog_cols],
    entity_effects=True,
    time_effects=True,
)
res = mod.fit(cov_type="clustered", cluster_entity=True)
print("\n=== PSTR Fixed-Effects (linear part) ===")
print(res)

print("\n=== MARGINAL EFFECTS OF TARIFF EXPOSURE (PSTR) ===")
open_levels = {
    "Low":    df["Openness_t1"].quantile(0.25),
    "Median": df["Openness_t1"].quantile(0.50),
    "High":   df["Openness_t1"].quantile(0.75),
}

alpha0 = res.params["Exposure_t"]
alpha1 = res.params["Exposure_t1"]
beta0  = res.params["Exp_t_G"]
beta1  = res.params["Exp_t1_G"]
V = res.cov

for label, q_orig in open_levels.items():
    z = (q_orig - open_mean) / open_std
    Gq = G_logistic(z, gamma_hat, c_hat_std)

    me_t = alpha0 + beta0 * Gq
    var_t = (
        V.loc["Exposure_t", "Exposure_t"]
        + 2*Gq*V.loc["Exposure_t", "Exp_t_G"]
        + (Gq**2)*V.loc["Exp_t_G", "Exp_t_G"]
    )
    se_t = float(np.sqrt(var_t))

    me_t1 = alpha1 + beta1 * Gq
    var_t1 = (
        V.loc["Exposure_t1", "Exposure_t1"]
        + 2*Gq*V.loc["Exposure_t1", "Exp_t1_G"]
        + (Gq**2)*V.loc["Exp_t1_G", "Exp_t1_G"]
    )
    se_t1 = float(np.sqrt(var_t1))

    cov_cross = (
        V.loc["Exposure_t", "Exposure_t1"]
        + Gq*(V.loc["Exposure_t", "Exp_t1_G"] + V.loc["Exp_t_G", "Exposure_t1"])
        + (Gq**2)*V.loc["Exp_t_G", "Exp_t1_G"]
    )
    var_cum = var_t + var_t1 + 2*cov_cross
    se_cum = float(np.sqrt(var_cum))
    me_cum = me_t + me_t1

    print(f"\n{label} Openness (orig={q_orig:.4f}, z={z:.3f}, G={Gq:.3f}):")
    print(f"  ∂y/∂Exposure_t      : {me_t: .6f}  (se = {se_t: .6f})")
    print(f"  ∂y/∂Exposure_(t-1)  : {me_t1:.6f}  (se = {se_t1:.6f})")
    print(f"  Cumulative (0–1 mo) : {me_cum:.6f}  (se = {se_cum:.6f})")



=== PSTR (NLS) transition estimates ===
  gamma (smoothness):  11.4330
  c (threshold, standardized): -1.1080
  c (threshold, ORIGINAL openness units): -0.1105

=== PSTR Fixed-Effects (linear part) ===
                          PanelOLS Estimation Summary                           
Dep. Variable:                      y   R-squared:                        0.0393
Estimator:                   PanelOLS   R-squared (Between):             -5.2395
No. Observations:                 260   R-squared (Within):              -0.0053
Date:                Sat, Nov 08 2025   R-squared (Overall):             -0.2835
Time:                        13:22:22   Log-likelihood                   -1088.4
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.4926
Entities:                          26   P-value                           0.1818
Avg Obs:                      10.0000   Distribution:               

## 7) Extended Specification — PSTR Model with Smooth Transition by Trade Openness

We now allow the effect of tariff exposure to vary **smoothly** with the level of trade openness given by the World Bank representing the global openess of a country.
Instead of assuming a linear interaction, we introduce a **logistic transition function** that captures gradual changes in the impact of exposure as openness increases.

The logistic transition function is defined as:

$$
G\!\left(\text{Openness}_{i,t-1}^{WB};\gamma,c\right)
= \frac{1}{1 + \exp\!\big[-\gamma\big(\text{Openness}_{i,t-1}^{WB} - c\big)\big]}.
$$

The corresponding PSTR regression is:

$$
\begin{aligned}
y_{i,t} &= \mu_i + \lambda_t
+ \alpha_0\,\text{Exposure}_{i,t}
+ \alpha_1\,\text{Exposure}_{i,t-1} \\
&\quad + \big(\beta_0\,\text{Exposure}_{i,t} + \beta_1\,\text{Exposure}_{i,t-1}\big)
\,G\!\left(\text{Openness}_{i,t-1}^{WB};\gamma,c\right)
+ \Gamma' Z_{i,t}
+ \varepsilon_{i,t}
\end{aligned}
$$

**Where:**

- $y_{i,t}$ — Outcome variable (e.g. Industrial Production YoY for country *i* in month *t*)
- $\text{Exposure}_{i,t}$ — Effective tariff exposure in month *t*
- $\text{Exposure}_{i,t-1}$ — One-month lag of exposure
- $\text{Openness}_{i,t-1}^{WB}$ — Lagged trade openness index (annual, mapped to months)
- $G(\text{Openness}_{i,t-1}^{WB};\gamma,c)$ — Logistic transition function with:
  - $c$: threshold (location) where $G=0.5$
  - $\gamma$: smoothness parameter controlling how sharp the transition is
- $Z_{i,t}$ — Monthly domestic controls (HICP YoY, Δ unemployment)
- $\mu_i$ — Country fixed effects
- $\lambda_t$ — Month fixed effects
- $\varepsilon_{i,t}$ — Error term clustered by country

---

### **Interpretation**

- $\alpha_0$, $\alpha_1$ — Baseline exposure effects when openness is low ($G \approx 0$)
- $\beta_0$, $\beta_1$ — Incremental effects as openness increases ($G \to 1$)
- **Marginal effects of exposure:**
  $$
  \frac{\partial y_{i,t}}{\partial \text{Exposure}_{i,t}}
  = \alpha_0 + \beta_0\,G(\text{Openness}_{i,t-1}^{WB};\gamma,c),
  $$
  $$
  \frac{\partial y_{i,t}}{\partial \text{Exposure}_{i,t-1}}
  = \alpha_1 + \beta_1\,G(\text{Openness}_{i,t-1}^{WB};\gamma,c).
  $$
- **Cumulative short-run effect (0–1 month):**
  $$
  (\alpha_0 + \alpha_1) + (\beta_0 + \beta_1)\,G(\text{Openness}_{i,t-1}^{WB};\gamma,c).
  $$
- When $\gamma$ is large, $G(\cdot)$ approximates a sharp threshold model; when small, the transition is smooth.

---

### **Estimation Details**

- **Estimator:** Nonlinear least squares with two-way (country and month) fixed effects
- **Frequency:** Monthly (2024-10 → 2025-09)
- **Standard Errors:** Clustered by country
- **Sample:** EU countries only
- **Controls:** Domestic macro variables (HICP YoY, unemployment)
- **Pre-processing:** Standardize openness before estimation; report the threshold $c$ in original units after transformation

---

### **Objective**

This model identifies whether the **effect of U.S. tariff shocks on economic outcomes** depends on how open each country is to international trade.
The logistic transition function captures a **nonlinear response** — for example, more open economies may only become significantly affected **after** crossing a critical openness threshold.


In [7]:
# Traditional Trade Openness
BASE = Path.cwd() / "data"

ipi        = pd.read_csv(BASE / "dependent_variable" / "IndustrialProductionIndex_df.csv")
stocks     = pd.read_csv(BASE / "dependent_variable" / "StockIndex_df.csv")
exposure   = pd.read_csv(BASE / "independent_variable" / "CountryTariffExposure_df.csv")
controls   = pd.read_csv(BASE / "control_variable" / "CountryControls_df.csv")
openness   = pd.read_csv(BASE / "transition_variable" / "WBTradeOpennessAnnual_df.csv")

for df in (ipi, stocks, exposure, controls):
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce").dt.to_period("M")

openness = openness[["Country", "Time", "Global Trade Openness-Lag1"]].copy()
openness.rename(columns={"Global Trade Openness-Lag1": "Openness_annual"}, inplace=True)
openness["Year"] = pd.to_datetime(openness["Time"].astype(str), errors="coerce").dt.year
openness = openness.drop(columns=["Time"])

monthly_open = []
for year in [2024, 2025]:
    months = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq="MS")
    tmp = pd.DataFrame({"Time_dt": months})
    tmp["Time"] = tmp["Time_dt"].dt.to_period("M")
    tmp = tmp.merge(openness[openness["Year"] == year][["Country", "Openness_annual"]], how="cross")
    monthly_open.append(tmp)

openness_monthly = pd.concat(monthly_open, ignore_index=True)
openness_monthly = openness_monthly[["Country", "Time", "Openness_annual"]]
openness_monthly = openness_monthly.rename(columns={"Openness_annual": "Openness"})

#df_dep = stocks.rename(columns={"Log Monthly Return": "y"})
df_dep = ipi.rename(columns={"diff_IPI": "y"})

exp = exposure.rename(columns={"Exposure": "Exposure_t"}).sort_values(["Country", "Time"])
exp["Exposure_t1"] = exp.groupby("Country")["Exposure_t"].shift(1)

exp = exp.merge(openness_monthly, on=["Country", "Time"], how="left")
exp["Openness_t1"] = exp.groupby("Country")["Openness"].shift(1)

exp = exp.dropna(subset=["Exposure_t1", "Openness_t1"])

df = (
    df_dep[["Country", "Time", "y"]]
    .merge(exp[["Country", "Time", "Exposure_t", "Exposure_t1", "Openness_t1"]],
           on=["Country", "Time"], how="inner")
    .merge(controls[["Country", "Time",
                     "HICP_lag1",
                     "Unemployment_lag1"]],
           on=["Country", "Time"], how="inner")
)

df = df[(df["Time"] >= pd.Period("2024-11", "M")) &
        (df["Time"] <= pd.Period("2025-08", "M"))].copy()

df["Time_dt"] = df["Time"].dt.start_time
df = df.set_index(["Country", "Time_dt"]).sort_index()
df.drop(columns=["Time"], inplace=True)

open_mean = df["Openness_t1"].mean()
open_std  = df["Openness_t1"].std(ddof=0)
df["Open_std"] = (df["Openness_t1"] - open_mean) / open_std

df = df.rename(columns={
    "HICP_lag1": "HICP",
    "Unemployment_lag1": "Unemp"
})

def twoway_within_transform(panel_df, cols):
    """
    Double-demean columns (entity & time FE).
    Returns a new DataFrame with *_wt (within-transformed) columns.
    """
    out = panel_df.copy()
    ent_means = out.groupby(level=0)[cols].transform("mean")
    time_means = out.groupby(level=1)[cols].transform("mean")
    overall_means = out[cols].mean()
    wt = out[cols] - ent_means - time_means + overall_means
    wt.columns = [c + "_wt" for c in cols]
    return wt

base_cols = ["y", "Exposure_t", "Exposure_t1", "HICP", "Unemp", "Open_std"]
wt = twoway_within_transform(df, base_cols)
for c in wt.columns:
    df[c] = wt[c]

def G_logistic(z, gamma, c):
    return 1.0 / (1.0 + np.exp(-gamma * (z - c)))

def build_X_within(gamma, c, work_df):
    """
    Build within-transformed X for given (γ,c):
      [ Exp_t_wt, Exp_t1_wt, (Exp_t*G)_wt, (Exp_t1*G)_wt, HICP_wt, Unemp_wt ]
    """
    z = work_df["Open_std_wt"].to_numpy()
    G = G_logistic(z, gamma, c)
    X = np.column_stack([
        work_df["Exposure_t_wt"].to_numpy(),
        work_df["Exposure_t1_wt"].to_numpy(),
        work_df["Exposure_t_wt"].to_numpy()  * G,
        work_df["Exposure_t1_wt"].to_numpy() * G,
        work_df["HICP_wt"].to_numpy(),
        work_df["Unemp_wt"].to_numpy(),
    ])
    return X

y_wt = df["y_wt"].to_numpy()

def ssr_objective(theta):
    gamma, c = theta
    if gamma <= 0:
        return 1e12 + (abs(gamma) + 1.0) * 1e12
    X = build_X_within(gamma, c, df)
    beta_hat, *_ = np.linalg.lstsq(X, y_wt, rcond=None)
    resid = y_wt - X @ beta_hat
    return float(resid @ resid)

theta0 = np.array([1.0, 0.0])
bounds = [(1e-3, 100.0), (-3.0, 3.0)]

opt = minimize(ssr_objective, theta0, method="L-BFGS-B", bounds=bounds)
gamma_hat, c_hat_std = opt.x
print("\n=== PSTR (NLS) transition estimates ===")
print(f"  gamma (smoothness): {gamma_hat: .4f}")
print(f"  c (threshold, standardized): {c_hat_std: .4f}")
c_hat_orig = open_mean + open_std * c_hat_std
print(f"  c (threshold, ORIGINAL openness units): {c_hat_orig: .4f}")

G_hat = G_logistic(df["Open_std"].to_numpy(), gamma_hat, c_hat_std)
df["G_hat"] = G_hat

df["Exp_t_G"]   = df["Exposure_t"]  * df["G_hat"]
df["Exp_t1_G"]  = df["Exposure_t1"] * df["G_hat"]

exog_cols = ["Exposure_t", "Exposure_t1", "Exp_t_G", "Exp_t1_G", "HICP", "Unemp"]

mod = PanelOLS(
    dependent=df["y"],
    exog=df[exog_cols],
    entity_effects=True,
    time_effects=True,
)
res = mod.fit(cov_type="clustered", cluster_entity=True)
print("\n=== PSTR Fixed-Effects (linear part) ===")
print(res)

print("\n=== MARGINAL EFFECTS OF TARIFF EXPOSURE (PSTR) ===")
open_levels = {
    "Low":    df["Openness_t1"].quantile(0.25),
    "Median": df["Openness_t1"].quantile(0.50),
    "High":   df["Openness_t1"].quantile(0.75),
}

alpha0 = res.params["Exposure_t"]
alpha1 = res.params["Exposure_t1"]
beta0  = res.params["Exp_t_G"]
beta1  = res.params["Exp_t1_G"]
V = res.cov

for label, q_orig in open_levels.items():
    z = (q_orig - open_mean) / open_std
    Gq = G_logistic(z, gamma_hat, c_hat_std)

    me_t = alpha0 + beta0 * Gq
    var_t = (
        V.loc["Exposure_t", "Exposure_t"]
        + 2*Gq*V.loc["Exposure_t", "Exp_t_G"]
        + (Gq**2)*V.loc["Exp_t_G", "Exp_t_G"]
    )
    se_t = float(np.sqrt(var_t))

    me_t1 = alpha1 + beta1 * Gq
    var_t1 = (
        V.loc["Exposure_t1", "Exposure_t1"]
        + 2*Gq*V.loc["Exposure_t1", "Exp_t1_G"]
        + (Gq**2)*V.loc["Exp_t1_G", "Exp_t1_G"]
    )
    se_t1 = float(np.sqrt(var_t1))

    cov_cross = (
        V.loc["Exposure_t", "Exposure_t1"]
        + Gq*(V.loc["Exposure_t", "Exp_t1_G"] + V.loc["Exp_t_G", "Exposure_t1"])
        + (Gq**2)*V.loc["Exp_t_G", "Exp_t1_G"]
    )
    var_cum = var_t + var_t1 + 2*cov_cross
    se_cum = float(np.sqrt(var_cum))
    me_cum = me_t + me_t1

    print(f"\n{label} Openness (orig={q_orig:.4f}, z={z:.3f}, G={Gq:.3f}):")
    print(f"  ∂y/∂Exposure_t      : {me_t: .6f}  (se = {se_t: .6f})")
    print(f"  ∂y/∂Exposure_(t-1)  : {me_t1:.6f}  (se = {se_t1:.6f})")
    print(f"  Cumulative (0–1 mo) : {me_cum:.6f}  (se = {se_cum:.6f})")



=== PSTR (NLS) transition estimates ===
  gamma (smoothness):  31.3052
  c (threshold, standardized): -0.5959
  c (threshold, ORIGINAL openness units):  98.2635

=== PSTR Fixed-Effects (linear part) ===
                          PanelOLS Estimation Summary                           
Dep. Variable:                      y   R-squared:                        0.0407
Estimator:                   PanelOLS   R-squared (Between):             -5.8818
No. Observations:                 260   R-squared (Within):               0.0060
Date:                Sat, Nov 08 2025   R-squared (Overall):             -0.3069
Time:                        13:24:48   Log-likelihood                   -1088.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.5473
Entities:                          26   P-value                           0.1640
Avg Obs:                      10.0000   Distribution:              

## 8) Extended Specification — PSTR Model with Smooth Transition by the proportion of Export with EU in the EU total intra market Export

We now allow the effect of tariff exposure to vary **smoothly** with the level of trade openness toward EU partner as a mesure of the resilience and the integration in the EU market.
Instead of assuming a linear interaction, we introduce a **logistic transition function** that captures gradual changes in the impact of exposure as openness increases.

The logistic transition function is defined as:

$$
G\!\left(\text{Resilience}_{i,t-1}^{EU};\gamma,c\right)
= \frac{1}{1 + \exp\!\big[-\gamma\big(\text{Resilience}_{i,t-1}^{EU} - c\big)\big]}.
$$

The corresponding PSTR regression is:

$$
\begin{aligned}
y_{i,t} &= \mu_i + \lambda_t
+ \alpha_0\,\text{Exposure}_{i,t}
+ \alpha_1\,\text{Exposure}_{i,t-1} \\
&\quad + \big(\beta_0\,\text{Exposure}_{i,t} + \beta_1\,\text{Exposure}_{i,t-1}\big)
\,G\!\left(\text{Resilience}_{i,t-1}^{EU};\gamma,c\right)
+ \Gamma' Z_{i,t}
+ \varepsilon_{i,t}
\end{aligned}
$$

**Where:**

- $y_{i,t}$ — Outcome variable (e.g. Industrial Production YoY for country *i* in month *t*)
- $\text{Exposure}_{i,t}$ — Effective tariff exposure in month *t*
- $\text{Exposure}_{i,t-1}$ — One-month lag of exposure
- $\text{Resilience}_{i,t-1}^{EU}$ — Lagged trade openness index (annual, mapped to months)
- $G(\text{Resilience}_{i,t-1}^{EU};\gamma,c)$ — Logistic transition function with:
  - $c$: threshold (location) where $G=0.5$
  - $\gamma$: smoothness parameter controlling how sharp the transition is
- $Z_{i,t}$ — Monthly domestic controls (HICP YoY, Δ unemployment)
- $\mu_i$ — Country fixed effects
- $\lambda_t$ — Month fixed effects
- $\varepsilon_{i,t}$ — Error term clustered by country

---

### **Interpretation**

- $\alpha_0$, $\alpha_1$ — Baseline exposure effects when openness is low ($G \approx 0$)
- $\beta_0$, $\beta_1$ — Incremental effects as openness increases ($G \to 1$)
- **Marginal effects of exposure:**
  $$
  \frac{\partial y_{i,t}}{\partial \text{Exposure}_{i,t}}
  = \alpha_0 + \beta_0\,G(\text{Resilience}_{i,t-1}^{EU};\gamma,c),
  $$
  $$
  \frac{\partial y_{i,t}}{\partial \text{Exposure}_{i,t-1}}
  = \alpha_1 + \beta_1\,G(\text{Resilience}_{i,t-1}^{EU};\gamma,c).
  $$
- **Cumulative short-run effect (0–1 month):**
  $$
  (\alpha_0 + \alpha_1) + (\beta_0 + \beta_1)\,G(\text{Resilience}_{i,t-1}^{EU};\gamma,c).
  $$
- When $\gamma$ is large, $G(\cdot)$ approximates a sharp threshold model; when small, the transition is smooth.

---

### **Estimation Details**

- **Estimator:** Nonlinear least squares with two-way (country and month) fixed effects
- **Frequency:** Monthly (2024-10 → 2025-09)
- **Standard Errors:** Clustered by country
- **Sample:** EU countries only
- **Controls:** Domestic macro variables (HICP YoY, unemployment)
- **Pre-processing:** Standardize openness before estimation; report the threshold $c$ in original units after transformation

---

### **Objective**

This model identifies whether the **effect of U.S. tariff shocks on economic outcomes** depends on how resilient / integrated each country is to international trade with Eu partner.
The logistic transition function captures a **nonlinear response** — for example, more resilient / integrated economies may only become significantly affected **after** crossing a critical openness threshold.


In [8]:
# ================================
# PSTR REGRESSION — FULLY WORKING
# ================================
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.optimize import minimize
from linearmodels.panel import PanelOLS

# -------------------------------
# 1. LOAD & CONVERT TIME
# -------------------------------
BASE = Path.cwd() / "data"
ipi = pd.read_csv(BASE / "dependent_variable" / "IndustrialProductionIndex_df.csv")
stocks = pd.read_csv(BASE / "dependent_variable" / "StockIndex_df.csv")
exposure = pd.read_csv(BASE / "independent_variable" / "CountryTariffExposure_df.csv")
controls = pd.read_csv(BASE / "control_variable" / "CountryControls_df.csv")
resilience = pd.read_csv(BASE / "transition_variable" / "EU_partner_index_df.csv")

# Convert ALL Time columns to Period[M]
for df in (ipi, stocks, exposure, controls, resilience):
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce").dt.to_period("M")

# -------------------------------
# 2. PREPARE DEPENDENT VARIABLE
# -------------------------------
#df_dep = stocks.rename(columns={"Log Monthly Return": "y"})[["Country", "Time", "y"]]
df_dep = ipi.rename(columns={"diff_IPI": "y"})

# -------------------------------
# 3. PREPARE EXPOSURE + LAG
# -------------------------------
exp = exposure.rename(columns={"Exposure": "Exposure_t"})
exp = exp.sort_values(["Country", "Time"]).reset_index(drop=True)
exp["Exposure_t1"] = exp.groupby("Country")["Exposure_t"].shift(1)

# -------------------------------
# 4. MONTHLY RESILIENCE (no spreading!)
# -------------------------------
resilience_monthly = (
    resilience[["Country", "Time", "OBS_VALUE_Lagged1"]]
    .rename(columns={"OBS_VALUE_Lagged1": "resilience"})
    .copy()
)

# Merge current resilience
exp = exp.merge(resilience_monthly, on=["Country", "Time"], how="left")

# Lag resilience
exp["resilience_t1"] = exp.groupby("Country")["resilience"].shift(1)

# Drop rows where lags are missing
exp = exp.dropna(subset=["Exposure_t1", "resilience_t1"]).reset_index(drop=True)

# -------------------------------
# 5. BUILD FINAL PANEL
# -------------------------------
df = (
    df_dep
    .merge(exp[["Country", "Time", "Exposure_t", "Exposure_t1", "resilience_t1"]],
           on=["Country", "Time"], how="inner")
    .merge(controls[["Country", "Time",
                     "HICP_lag1",
                     "Unemployment_lag1"]],
           on=["Country", "Time"], how="inner")
)

# Time window
df = df[(df["Time"] >= "2024-11") & (df["Time"] <= "2025-08")].copy()

# Standardize resilience for PSTR
open_mean = df["resilience_t1"].mean()
open_std = df["resilience_t1"].std(ddof=0)
df["Open_std"] = (df["resilience_t1"] - open_mean) / open_std

# Rename controls
df = df.rename(columns={
    "HICP_lag1": "HICP",
    "Unemployment_lag1": "Unemp"
})

# Set index for PanelOLS
df["Time_dt"] = df["Time"].dt.start_time
df = df.set_index(["Country", "Time_dt"]).sort_index()

# -------------------------------
# 6. TWO-WAY WITHIN TRANSFORM
# -------------------------------
def twoway_within(df, cols):
    ent = df.groupby(level=0)[cols].transform("mean")
    tim = df.groupby(level=1)[cols].transform("mean")
    overall = df[cols].mean()
    return df[cols] - ent - tim + overall

base_cols = ["y", "Exposure_t", "Exposure_t1", "HICP", "Unemp", "Open_std"]
wt = twoway_within(df, base_cols)
wt.columns = [c + "_wt" for c in wt.columns]   # <-- fixed typo: "*wt" → "_wt"
df = pd.concat([df, wt], axis=1)

# -------------------------------
# 7. PSTR: ESTIMATE γ and c
# -------------------------------
def G_logistic(z, gamma, c):
    return 1.0 / (1.0 + np.exp(-gamma * (z - c)))

def build_X(gamma, c, work):
    z = work["Open_std_wt"].to_numpy()
    G = G_logistic(z, gamma, c)
    X = np.column_stack([
        work["Exposure_t_wt"],
        work["Exposure_t1_wt"],
        work["Exposure_t_wt"] * G,
        work["Exposure_t1_wt"] * G,
        work["HICP_wt"],
        work["Unemp_wt"]
    ])
    return X

y_wt = df["y_wt"].to_numpy()

def ssr(theta):
    gamma, c = theta
    if gamma <= 0:
        return 1e12
    X = build_X(gamma, c, df)
    beta, *_ = np.linalg.lstsq(X, y_wt, rcond=None)   # <-- fixed: ** → *_
    return float((y_wt - X @ beta) @ (y_wt - X @ beta))

opt = minimize(ssr, x0=[1.0, 0.0], bounds=[(0.1, 100), (-3, 3)], method="L-BFGS-B")
gamma_hat, c_hat_std = opt.x
c_hat = open_mean + open_std * c_hat_std

print("\nPSTR TRANSITION ESTIMATES")
print(f"γ (smoothness) = {gamma_hat: .3f}")
print(f"c (threshold, original units) = {c_hat: .4f}")

# -------------------------------
# 8. FINAL LINEAR MODEL WITH G(z)
# -------------------------------
z = df["Open_std"].to_numpy()
G = G_logistic(z, gamma_hat, c_hat_std)
df["G"] = G
df["Exp_t_G"] = df["Exposure_t"] * G
df["Exp_t1_G"] = df["Exposure_t1"] * G

exog = df[["Exposure_t", "Exposure_t1", "Exp_t_G", "Exp_t1_G", "HICP", "Unemp"]]
mod = PanelOLS(dependent=df["y"], exog=exog, entity_effects=True, time_effects=True)
res = mod.fit(cov_type="clustered", cluster_entity=True)
print("\nPSTR LINEAR PART")
print(res)

# -------------------------------
# 9. MARGINAL EFFECTS
# -------------------------------
alpha0 = res.params["Exposure_t"]
alpha1 = res.params["Exposure_t1"]
beta0  = res.params["Exp_t_G"]
beta1  = res.params["Exp_t1_G"]
V      = res.cov

levels = {
    "Low":     df["resilience_t1"].quantile(0.25),
    "Median":  df["resilience_t1"].quantile(0.50),
    "High":    df["resilience_t1"].quantile(0.75)
}

print("\nMARGINAL EFFECTS")
for name, val in levels.items():
    zq = (val - open_mean) / open_std
    Gq = G_logistic(zq, gamma_hat, c_hat_std)
    me_t   = alpha0 + beta0 * Gq
    me_t1  = alpha1 + beta1 * Gq
    me_cum = me_t  + me_t1

    var_t = (V.loc["Exposure_t","Exposure_t"] +
             2*Gq*V.loc["Exposure_t","Exp_t_G"] +
             Gq**2*V.loc["Exp_t_G","Exp_t_G"])

    var_t1 = (V.loc["Exposure_t1","Exposure_t1"] +
              2*Gq*V.loc["Exposure_t1","Exp_t1_G"] +
              Gq**2*V.loc["Exp_t1_G","Exp_t1_G"])

    cov_cross = (V.loc["Exposure_t","Exposure_t1"] +
                 Gq*(V.loc["Exposure_t","Exp_t1_G"] + V.loc["Exp_t_G","Exposure_t1"]) +
                 Gq**2*V.loc["Exp_t_G","Exp_t1_G"])

    var_cum = var_t + var_t1 + 2*cov_cross

    print(f"\n{name} resilience ({val:.3f}) → G(z) = {Gq:.3f}")
    print(f" ∂y/∂Exp_t = {me_t: .5f} (se = {np.sqrt(var_t):.5f})")
    print(f" ∂y/∂Exp_(t-1) = {me_t1:.5f} (se = {np.sqrt(var_t1):.5f})")
    print(f" Cumulative = {me_cum:.5f} (se = {np.sqrt(var_cum):.5f})")


PSTR TRANSITION ESTIMATES
γ (smoothness) =  2.653
c (threshold, original units) =  0.1822

PSTR LINEAR PART
                          PanelOLS Estimation Summary                           
Dep. Variable:                      y   R-squared:                        0.0473
Estimator:                   PanelOLS   R-squared (Between):             -5.5509
No. Observations:                 208   R-squared (Within):              -0.0052
Date:                Sat, Nov 08 2025   R-squared (Overall):             -0.1528
Time:                        13:24:57   Log-likelihood                   -870.50
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.3993
Entities:                          26   P-value                           0.2176
Avg Obs:                       8.0000   Distribution:                   F(6,169)
Min Obs:                       8.0000                                           
